In [2]:
import PIL,torch
import matplotlib.pyplot as plt
import torch.nn as nn
import numpy as np
import os
from PIL import Image
from torchvision import transforms
from torch.utils.data import Dataset,DataLoader
from torchvision.models import densenet121

In [3]:
class SkinDataset(Dataset):
    folds=5
    def __init__(self, root , num_classes,fold=0,training=False,n_augment=2):
        self.data_path = []
        self.sides =(224,224)
        margin_sides = (256, 256)
        self.n_augment = min(n_augment , 4)
        if training:
            self.transform = transforms.Compose([
#                 transforms.RandomAffine(degrees=45),
                transforms.RandomResizedCrop(size=self.sides),
#                 transforms.RandomHorizontalFlip(p=0.5),
#                 transforms.RandomVerticalFlip(p=0.5),
                transforms.ToTensor(),
                transforms.Normalize(mean= [0.6075306,0.49116918 ,0.46066117],std = [0.22603881, 0.21623525, 0.2191065 ])
#                 transforms.Normalize()
            ])
        else:
            self.transform = transforms.Compose([
#                 transforms.RandomAffine(degrees=30),
#                 transforms.RandomResizedCrop(size=(224, 224)),
                
                transforms.Resize(margin_sides),
                transforms.CenterCrop(size = self.sides),
                transforms.ToTensor(),
                transforms.Normalize(mean= [0.6075306,0.49116918 ,0.46066117],std = [0.22603881, 0.21623525, 0.2191065 ])
#                 transforms.Normalize()
            ])
        self.training = training
        for label in range(num_classes):
            self.data_dir = os.path.join(root,os.listdir(root)[label])
            self.filename = os.listdir(self.data_dir)
            l = len(self.filename)
            inter = l//SkinDataset.folds
            picked = list(range(inter* fold,inter * (fold+1))) if not training else list(range(0,inter*fold))+list(range(inter*(fold+1),l))

            for i in picked:
                file_path = os.path.join(self.data_dir , self.filename[i])
                self.data_path.append((file_path, label))
    
    def __getitem__(self , index):
        index %= len(self.data_path)
        ddir , label = self.data_path[index]
        img = Image.open(ddir)
        img_size = img.size
        img = img.crop((0,0,img_size[0],img_size[1]-65))
        imgmat = self.transform(img)
        result = (imgmat, label)
        del imgmat
        del img
        return result
    
    def __len__(self):
        if self.training:
            return len(self.data_path)*self.n_augment
        else:
            return len(self.data_path)

In [4]:
batch_size = 16
training_dataset = SkinDataset(os.path.join('/home/jovyan/baseline_x/Skin40'), 40, training=True,n_augment=1)
test_dataset = SkinDataset(os.path.join('/home/jovyan/baseline_x/Skin40'), 40 , n_augment=1)
training_dataloader = DataLoader(training_dataset , batch_size = batch_size , num_workers = 1, shuffle = True)
test_dataloader = DataLoader(test_dataset , batch_size = batch_size , num_workers = 1, shuffle = True)

In [5]:
def train(model, train_loader, loss_func, optimizer, device):
    """
    train model using loss_fn and optimizer in an epoch.
    model: CNN networks
    train_loader: a Dataloader object with training data
    loss_func: loss function
    device: train on cpu or gpu device
    """
    total_loss = 0
    model.train()
    # train the model using minibatch
    for i, (images, targets) in enumerate(train_loader):
        images = images.to(device)
        targets = targets.to(device)

        # forward
        outputs = model(images)
        loss = loss_func(outputs, targets)

        # backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        if (i + 1) % (40) == 0:
            print ("Step [{}/{}] Train Loss: {:.4f}"
                   .format(i+1, len(train_loader), loss.item()))
    return total_loss / len(train_loader)

In [6]:
def evaluate(model, val_loader, device, nclasses):
    """
    model: CNN networks
    val_loader: a Dataloader object with validation data
    device: evaluate on cpu or gpu device
    return classification accuracy of the model on val dataset
    """
    # evaluate the model
    model.eval()
    # context-manager that disabled gradient computation
    with torch.no_grad():
        correct = 0
        total = 0
        cm = torch.zeros(nclasses, nclasses)
        for i, (images, targets) in enumerate(val_loader):
            # device: cpu or gpu
            images = images.to(device)
            targets = targets.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, dim=1)
            
            
            for t, p in zip(targets.view(-1), predicted.view(-1)):
                cm[t.long()][p.long()] +=1
            correct += (predicted == targets).sum().item()
            total += targets.size(0)
            
        accuracy = correct / total
        print('Accuracy on Test Set: {:.4f} %'.format(100 * accuracy))
        return accuracy

In [7]:
import matplotlib.pyplot as plt
def show_curve(ys, title):
    """
    plot curlve for Loss and Accuacy
    Args:
        ys: loss or acc list
        title: loss or accuracy
    """
    x = np.array(range(len(ys)))
    y = np.array(ys)
    plt.plot(x, y, c='b')
    plt.axis()
    plt.title('{} curve'.format(title))
    plt.xlabel('epoch')
    plt.ylabel('{}'.format(title))
    plt.show()

In [8]:
from datetime import datetime
def fit(model, num_epochs, optimizer, schedulr, device,nclasses):
    loss_func = nn.CrossEntropyLoss()
    model.to(device)
    loss_func.to(device)
    losses = []
    accs = []
    
    for epoch in range(num_epochs):
        
        print('Epoch {}/{}: , lr = {}'.format(epoch + 1, num_epochs , optimizer.param_groups[0]['lr']))
        # train step
        start_t=datetime.now()
        loss = train(model, training_dataloader, loss_func, optimizer, device)
        losses.append(loss)
        schedulr.step()
        
        # evaluate step
        accuracy = evaluate(model, test_dataloader, device,nclasses)
        accs.append(accuracy)

    show_curve(losses, "train loss")
    show_curve(accs, "test accuracy")

In [9]:
net = densenet121(pretrained=True)
net.classifier = torch.nn.Sequential(nn.Linear(net.classifier.in_features, 40)
                            )
'''net.classifier = torch.nn.Sequential(nn.Linear(net.classifier.in_features, 4096),
                                     nn.ReLU(inplace=True),
                                     nn.Dropout(0.5),
                                     nn.Linear(4096,40)
                            )'''
'''net.classifier = torch.nn.Sequential(nn.Linear(net.classifier.in_features, 4096),
                                     nn.ReLU(inplace=True),
                                     nn.Dropout(0.5),
                                     nn.Linear(4096,1024),
                                     nn.ReLU(inplace=True),
                                     nn.Dropout(0.5),
                                     nn.Linear(1024,40)
                            )'''
'''net.classifier = torch.nn.Sequential(nn.Linear(net.classifier.in_features, 4096),
                                     nn.ReLU(inplace=True),
                                     nn.Dropout(0.5),
                                     nn.Linear(4096,2048),
                                     nn.ReLU(inplace=True),
                                     nn.Dropout(0.5),
                                     nn.Linear(2048,1024),
                                     nn.ReLU(inplace=True),
                                     nn.Dropout(0.5),
                                     nn.Linear(1024,40)
                            )'''
for param in net.classifier.parameters():
    if len(param.shape)>1:
        torch.nn.init.xavier_normal_(param)

In [10]:
# Hyper-parameters
num_epochs = 8
lr = 1e-4
nclasses = 40
feature_tune=False
# Device configuration
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


for parma in net.parameters():
    parma.requires_grad = feature_tune
for param in net.classifier.parameters():
    param.requires_grad = True
    
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, net.parameters()), lr=lr,weight_decay=1e-3)
schedulr = torch.optim.lr_scheduler.StepLR(optimizer , step_size = 1 , gamma = 0.5)

In [12]:
fit(net, num_epochs, optimizer, schedulr,device, nclasses)  #

Epoch 1/8: , lr = 0.0001


KeyboardInterrupt: 

In [ ]:
evaluate(net, test_dataloader, device,nclasses)